In [1]:
!pip install bertopic
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np
from sklearn.naive_bayes import GaussianNB
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

KeyboardInterrupt: 

In [ ]:
transform = transforms.Compose([ #normalized data, can be done without normalized data but threshold will need to be changed
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

In [ ]:
def regions(image, size): #divide image into non overlapping regions of specified size
    image.numpy()
    regions = []
    for i in range(0, 28, size): 
        for j in range(0, 28, size):
            region = image[0, i:i+size, j:j+size]
            regions.append(region)
    return regions

def encode(region, threshold):
    region = torch.where(region < threshold, 0, 1)  # Vectorized thresholding
    binary_str = ''.join(map(str, region.flatten().int().tolist()))  # Convert to binary
    return int(binary_str, 2)  # Convert binary to decimal

In [ ]:
region_size = 7 # for 4x4 region max binary value is 65535
threshold = -0.5

encoded_images = []
for image, label in train_dataset: #with the for loops, took my laptop around 30 min to run on 4x4
    r = regions(image, region_size)
    for i in range(len(r)):
        r[i] = encode(r[i], threshold)
    encoded_images.append(r) #in theory this encodes all the regions and stores them. each image is its own list of decimal regions
    
for i in range(len(encoded_images)):
    encoded_images[i] = [x for x in encoded_images[i] if x != 0] #remove any regions that are just 0 (all black/no important features according to threshold)

In [ ]:
text_data = [' '.join(map(str,encoded_images[i])) for i in range(len(encoded_images))] #joins the regions together into one string instead of a list of strings

traindata = {'Document':text_data, 'Label':train_dataset.targets.tolist()} #dataframe of images and labels
traindf = pd.DataFrame(traindata)
print('LENGTH: ', len(traindf))

groupeddf = traindf.groupby('Label', as_index=False).agg({'Document': ''.join}) #dataframe of documents grouped by class (all documents of one class are combined into a mega document)
print('LENGTH: ', len(groupeddf))
nums = groupeddf['Document']
labels = groupeddf['Label']

In [ ]:
groupeddf.head()

In [ ]:
#cvectorizer = CountVectorizer()
#count = cvectorizer.fit_transform(features)
#nums = cvectorizer.get_feature_names_out()

ctfidf, features = BERTopic()._c_tf_idf(groupeddf, fit=True) #class based TFIDF
ctfidf_array = ctfidf.toarray()

for idx, topic in enumerate(groupeddf['Label']): #print the top 10 words for each class
    print(f"Top words for Class {topic}:")
    top_indices = ctfidf_array[idx].argsort()[-10:][::-1]
    top_words = [features[i] for i in top_indices]
    print(top_words)
    
#For whatever reason it pulls in the same number as the highest rated number for all classes
#Can we just scrap those or is it a broader issue and are the rest of the numbers wrong?
#Same thing happened when the model was broken and was only 1s, it had all 1s preceded by the class number
#Here it is all 0s preceded by the class number
#also maybe need to go even bigger on the region based on the fact that the highest rated number for 3 classes is a white brick of all 1s